# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), True)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-950000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/righty_suit-

### Running through an example

In [2]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = True, True

# You sit North and hold:
hand = 'J.32.KJ983.QJT74'

auction = ["PAD_START","2C"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, 0, True)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

Setting seed (Sampling bidding info) from J.32.KJ983.QJT74: 2930407879
bid X value 0.6273 is recommended by NN
CandidateBid(bid=X   , insta_score=0.6273, expected_score=---, adjust=---)
CandidateBid(bid=PASS, insta_score=0.3717, expected_score=---, adjust=---)
Sampling for aution: ['PAD_START', '2C'] trying to find 5000
sample_cards_auction, nsteps= 1
NS:  1.0 EW:  99.0 Auction:  ['PAD_START', '2C']
nesw_i 0
Player:  S Hand:  J.xx.KJ98x.QJTxx
HCP:  [[ 6.4   4.88 21.  ]]
Shape:  [[3.94 3.68 2.76 2.56 3.72 3.52 2.91 2.79 4.09 3.69 2.76 2.41]]
sample_cards_vec generating 5000
sample_cards_vec took 0.09768 Deals: 2977
n_samples 2977 matching bidding info
Samples after bidding filtering:  2977  Threshold:  0.7
 X    0.627 Samples: 200
bidding_rollout - n_samples:  200
bidding_rollout - finished  (200, 64)
[  -96.39  -156.22  -593.53  -505.28  -253.41  -395.31  -197.99  -681.61  -116.64  -231.95  -206.76  -827.18  -222.74  -236.71  -834.33   493.39  -239.95  -685.6   -239.72   125.92  -164.0

[{'call': 'PASS',
  'insta_score': 0.372,
  'expected_score': -42,
  'adjustment': 19},
 {'call': 'X', 'insta_score': 0.627, 'expected_score': -301, 'adjustment': 31}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [3]:
### Samples consistent with the auction so far

for i in range(min(10,len(bid.samples))):
    print(bid.samples[i])


J.xx.KJ98x.QJTxx Qxxx.KJxx.xx.K8x 98xx.Txx.AQTx.xx AKTx.AQ98.xx.A9x 0.75000 
 2C-X-2N-3C-X-P-4N-P-5D-P-5H-P-P-P (10.1, 10.9)  
 2C-P-2N-P-3C-P-3H-P-4H-P-4N-P-5D-P-5N-P-6D-P-6H-P-P-P (10.1, 11.0) 
J.xx.KJ98x.QJTxx KTxxx.JT9.xxx.8x Axx.Q8xx.Txx.K9x Q98x.AKxx.AQ.Axx 0.75009 
 2C-X-2N-3C-P-3H-3S-P-P-P (9.8, 9.7)  
 2C-P-2N-P-3H-P-4N-P-5D-P-5S-P-P-P (9.8, 9.7) 
J.xx.KJ98x.QJTxx Txx.AKT9xx.xx.8x K9xx.QJ.QTx.9xxx AQ8xx.8xx.Axx.AK 0.75001 
 2C-X-2S-3C-P-3S-4S-P-P-P (10.9, 9.8)  
 2C-P-2S-P-3C-P-4N-P-5D-P-5N-P-6D-P-6N-P-P-P (9.8, 9.6) 
J.xx.KJ98x.QJTxx K8xxx.K9xx.Txx.x 9xxx.QT.Qxxx.A9x AQT.AJ8xx.A.K8xx 0.75012 
 2C-X-2H-3C-3H-P-4N-P-5D-P-5H-P-P-P (11.4, 11.3)  
 2C-P-2H-P-3H-P-4N-P-5D-P-5S-P-6H-P-P-P (11.4, 11.3) 
J.xx.KJ98x.QJTxx Qxxx.AT9xx.xx.xx KTxx.QJx.Qxx.8xx A98x.K8x.ATx.AK9 0.75001 
 2C-X-2N-3C-P-3S-3N-P-P-P (8.6, 8.7)  
 2C-P-2N-P-3C-P-4N-P-5D-P-5H-P-P-P (9.8, 9.5) 
J.xx.KJ98x.QJTxx KT9xx.Jx.Qxx.8xx 8xxx.AK8xx.T.9xx AQx.QT9x.Axxx.AK 0.75003 
 2C-X-2N-3C-P-3H-3N-P-P-P (9.1, 8.7)  
 2C-P-

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [4]:
# we got some samples above. if we want more, we can always get more

auction = ["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"]
sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(auction,0)
print("Good quality:", good_quality)
for i in range(min(10,sample_hands.shape[0])):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])} {sorted_score[i]:.2f}')

sample_cards_auction, nsteps= 3
NS:  1.0 EW:  99.0 Auction:  ['PAD_START', 'PAD_START', 'PAD_START', 'PASS', '1N', 'PASS', '3S', 'PASS']
nesw_i 0
Player:  N Hand:  J.xx.KJ98x.QJTxx
HCP:  [[ 7.98 16.72  7.62]]
Shape:  [[2.58 4.16 3.25 3.06 6.29 2.82 1.93 1.87 2.74 3.91 3.21 3.13]]
sample_cards_vec generating 5000
sample_cards_vec took 0.112 Deals: 2430
n_samples 5000 matching bidding info
Samples after bidding filtering:  0  Threshold:  0.7
Only found 36 10
Good quality: True
J.xx.KJ98x.QJTxx KT9xx.Q98x.xxx.x AQxx.AKx.Axx.AKx 8xx.JTxx.QT.98xx 0.50
J.xx.KJ98x.QJTxx QTxxx.Txx.Tx.xxx A8xx.AK8.AQx.AK8 K9x.QJ9xx.xxx.9x 0.50
J.xx.KJ98x.QJTxx K9xxx.K9x.QTx.xx AQT.AJTx.xx.A9xx 8xxx.Q8xx.Axx.K8 0.50
J.xx.KJ98x.QJTxx Txx.98.Axxxx.A8x AK98xx.AKQTxx..x Qxx.Jxx.QTx.K9xx 0.50
J.xx.KJ98x.QJTxx 9xx.KJT8xx.Tx.xx AKTx.AQx.Ax.AK8x Q8xxx.9x.Qxxx.9x 0.50
J.xx.KJ98x.QJTxx Txxx.K9xx.Txx.xx KQ9x.AJ8.AQxx.AK A8xx.QTxx.x.98xx 0.50
J.xx.KJ98x.QJTxx QTxx.T9x.Axx.A9x AK98xx.AKQ8xx.x. xx.Jx.QTxx.K8xxx 0.50
J.xx.KJ98